In [231]:
from typing import Dict, Optional
from datasets import Dataset, load_dataset
import os
import glob
import json
import copy
import threading
import time
import random
from typing import Dict, Optional
import requests
import numpy as np

import fasttext
lang_detect = fasttext.load_model('../fastchat/modules/fasttext/lid.176.bin')

from fastchat.modules.answer_refiner import generate_refiner

In [ ]:
# from peft import LoraConfig
# from transformers import AutoModelForCausalLM, AutoTokenizer, HfArgumentParser, TrainingArguments

# from trl import DPOTrainer

def extract_anthropic_prompt(prompt_and_response, search_term="\n\nAssistant:"):
    """Extract the anthropic prompt from a prompt and response pair."""
    search_term_idx = prompt_and_response.rfind(search_term)
    assert search_term_idx != -1, f"Prompt and response does not contain '{search_term}'"
    return prompt_and_response[: search_term_idx + len(search_term)]


def get_hh(split: str, sanity_check: bool = False, silent: bool = False, cache_dir: str = None) -> Dataset:
    """Load the Anthropic Helpful-Harmless dataset from Hugging Face and convert it to the necessary format.

    The dataset is converted to a dictionary with the following structure:
    {
        'prompt': List[str],
        'chosen': List[str],
        'rejected': List[str],
    }

    Prompts should be structured as follows:
      \n\nHuman: <prompt>\n\nAssistant:
    Multiple turns are allowed, but the prompt should always start with \n\nHuman: and end with \n\nAssistant:.
    """
    dataset = load_dataset("Anthropic/hh-rlhf", split=split, cache_dir=cache_dir)
    if sanity_check:
        dataset = dataset.select(range(min(len(dataset), 1000)))

    def split_prompt_and_responses(sample) -> Dict[str, str]:
        prompt = extract_anthropic_prompt(sample["chosen"])
        return {
            "prompt": prompt,
            "chosen": sample["chosen"][len(prompt) :],
            "rejected": sample["rejected"][len(prompt) :],
        }

    return dataset.map(split_prompt_and_responses)

In [ ]:
split = 'train'
cache_dir = None
dataset = load_dataset("Anthropic/hh-rlhf", split=split, cache_dir=cache_dir)

In [ ]:
from typing import Dict, Optional
from datasets import Dataset, load_dataset

from fastchat.model.model_adapter import get_conversation_template

def extract_anthropic_prompt(prompt_and_response, search_term="\n\nAssistant:"):
    """Extract the anthropic prompt from a prompt and response pair."""
    search_term_idx = prompt_and_response.rfind(search_term)
    assert search_term_idx != -1, f"Prompt and response does not contain '{search_term}'"
    return prompt_and_response[: search_term_idx + len(search_term)]

class hankang_DPODataset:
    def __init__(
        self, 
        dataset_path="/data/llm_datasets/Ultrafeedback_binarized.ko.hankang/",
        data_format='chat-orca',
        search_term='\n\n### Assistant:',
        num_train=None,
        num_eval=None,
    ):
        self.dataset_path = dataset_path
        self.data_format = data_format
        self.search_term = search_term
        self.num_train = num_train
        self.num_eval = num_eval
    
    def get_prompt_and_response(self, data):
        conv = get_conversation_template(self.data_format)

        for idx, _conv in enumerate(data):
            role = _conv['role']
            content = _conv['content_kr']
            if idx % 2 == 0 and role == 'user':
                conv.append_message(conv.roles[0], content)
            elif idx % 2 == 1 and role == 'assistant':
                conv.append_message(conv.roles[1], content)
            else:
                print("Warning: data type invaild")

        if len(conv.messages) == 0:
            print("Warning: data is empty")
        if len(conv.messages) % 2 != 0:
            print("Warning: data has weird pair")

        return conv.get_prompt()
    
    def make_dpo_data_module(self):
        def validate_prompt_and_responses(data) -> bool:
            try:
                prompt_and_response = self.get_prompt_and_response(data['chosen'])
                prompt_and_response_rejected = self.get_prompt_and_response(data['rejected'])
                prompt = extract_anthropic_prompt(prompt_and_response, self.search_term)
                promopt_rejected = extract_anthropic_prompt(prompt_and_response_rejected, self.search_term)
            except AssertionError:
                return False

            return True

        def split_prompt_and_responses(data) -> Dict[str, str]:
            prompt_and_response = self.get_prompt_and_response(data['chosen'])
            prompt_and_response_rejected = self.get_prompt_and_response(data['rejected'])
            prompt = extract_anthropic_prompt(prompt_and_response, self.search_term)
            promopt_rejected = extract_anthropic_prompt(prompt_and_response_rejected, self.search_term)
            return {
                "prompt": prompt,
                "chosen": prompt_and_response[len(prompt) :],
                "rejected": prompt_and_response_rejected[len(promopt_rejected) :],
            }
                             
                             
        dataset = load_dataset(self.dataset_path)

        train_dataset = dataset['train']
        eval_dataset = dataset['test']

        original_columns = list(train_dataset.features.keys())

        if self.num_train is not None:
            train_dataset = train_dataset.select(range(min(len(train_dataset), self.num_train)))
        if self.num_eval is not None:
            eval_dataset = eval_dataset.select(range(min(len(train_dataset), self.num_eval)))

        train_dataset = train_dataset.filter(validate_prompt_and_responses)
        train_dataset = train_dataset.map(split_prompt_and_responses, remove_columns=original_columns)

        eval_dataset = eval_dataset.filter(validate_prompt_and_responses)
        eval_dataset = eval_dataset.map(split_prompt_and_responses, remove_columns=original_columns)

        return dict(train_dataset=train_dataset, eval_dataset=eval_dataset)

In [ ]:
from fastchat.train.data_modules.dpo_dataset import hankang_DPODataset

dpo_dataset = hankang_DPODataset()
dpo_datamodule = dpo_dataset.make_dpo_data_module()

In [ ]:
eval_dataset = dpo_datamodule['eval_dataset']

In [ ]:
eval_dataset[0]

In [ ]:
eval_dataset = dedup_dataset(eval_dataset, 'chat-orca')

In [ ]:
eval_dataset[0]

In [ ]:
import chromadb
from chromadb.config import Settings
import random
from fastchat.modules.embedder_adapter import Embedder, get_embedder
from fastchat.conversation import (
    SeparatorStyle,
)
from fastchat.model.model_adapter import get_conversation_template
import copy

# def dedup_by_similarity(dataset, prompt_template='chat-orca', target_text_len=100, n_results=100, distance_threshold = 0.6):
_dataset = train_dataset
prompt_template='vicuna'
target_text_len=100
n_results=100
distance_threshold = 0.35
    
if prompt_template == 'chat-orca':
    conv = get_conversation_template(prompt_template)
    system_message = conv.system_message
    sep_style = conv.sep_style
    sep = conv.sep
    prompt_user, prompt_bot = conv.roles

    len_sep_style = 0
    if sep_style == SeparatorStyle.ADD_COLON_TWO:
        len_sep_style = 1

    len_front = len(system_message) + len(sep) + len(prompt_user) + len_sep_style + 1
    len_rear = len(sep) + len(prompt_bot) + len_sep_style
    def filter_question(data):
        return { 
            # **data,
            'prompt': data['prompt'][len_front:-len_rear][:target_text_len]
        }

if prompt_template == 'vicuna':
    def filter_question(data):
        return {
            'prompt': data['conversations'][0]['value'][:target_text_len]
        }

question_dataset = _dataset.map(filter_question)

chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
embedder = get_embedder("ddobokki/klue-roberta-base-nli-sts-ko-en")
collection = chroma_client.create_collection(name="context", embedding_function=embedder.embed, metadata={"hnsw:space": "cosine"})
ids = []
# add
texts = question_dataset['prompt']
last_id = -1
new_ids = [f"id{i+last_id+1}" for i in range(len(texts))]
ids += new_ids
collection.add(documents=texts, ids=new_ids)

query_ids = copy.deepcopy(new_ids)
selected_ids = []
duplicated_ids = []

weird_ids = []
error_ids = []
while query_ids:
    current_id = random.choice(query_ids)
    if current_id in selected_ids:
        print("Warning: this is weird..")
        weird_ids.append(current_id)
        continue
    selected_ids.append(current_id)
    search_strings = [texts[int(current_id[2:])]]
    if collection.count() == 0:
        print("Warning: collection is empty. Forced break")
        break
    result = collection.query(query_texts=search_strings, n_results=min(n_results, len(query_ids)), include=['distances']) #'documents'

    search_ids = result['ids'][0]
    distances = result['distances'][0]
    remove_ids = []
    for idx in range(len(search_ids)):
        sid = search_ids[idx]
        dist = distances[idx]
        if dist < distance_threshold:
            remove_ids.append(sid)

    for rid in remove_ids:
        if rid in query_ids:
            query_ids.remove(rid)
            
    if remove_ids:
        duplicated_ids += remove_ids
        collection.delete(ids=remove_ids)
    else:
        print("Warning: this is error..")
        error_ids.append(current_id)

    print(f"Total:{len(new_ids)} Selected:{len(selected_ids)} current_dup:{len(remove_ids)} vector_store:{collection.count()} remained:{len(query_ids)} total_dup:{len(duplicated_ids)}", '\t\t\t\t\t', end='\r')

print('finished dedup data:', f"Total:{len(new_ids)} Selected:{len(selected_ids)} current_dup:{len(remove_ids)} vector_store:{collection.count()} remained:{len(query_ids)} total_dup:{len(duplicated_ids)}")

selected_ids = [int(sid[2:]) for sid in set(selected_ids)]

_dataset = _dataset.select(selected_ids)

# return dataset, selected_ids, query_ids

In [588]:
current_id

'id4377'

In [590]:
selected_ids[-3:]

[23397, 16450, 1072]

In [576]:
search_strings

['목록에서 1번을 어떻게 할 수 있을까요?']

In [578]:
collection.delete(

0

In [594]:
chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
embedder = get_embedder("ddobokki/klue-roberta-base-nli-sts-ko-en")
collection = chroma_client.create_collection(name="context", embedding_function=embedder.embed, metadata={"hnsw:space": "cosine"})
ids = []
# add
texts = question_dataset['prompt']
last_id = -1
new_ids = [f"id{i+last_id+1}" for i in range(len(texts))]
ids += new_ids
collection.add(documents=texts, ids=new_ids)

In [597]:
result = collection.query(query_texts='목록에서 1번을 어떻게 할 수 있을까요?', n_results=min(n_results, len(query_ids)), include=['distances', 'documents']) #

search_ids = result['ids'][0]
distances = result['distances'][0]
documents = result['documents'][0]

In [599]:
collection.count()

32373

In [587]:
train_dataset[1072]

{'conversations': [{'from': 'human', 'value': '아래 녹취록을 요약해 주세요.'},
  {'from': 'gpt',
   'value': '물론입니다! 제공해주신 녹취록을 요약하면 다음과 같습니다:\n\n연사는 최근 실시된 직원 참여도에 관한 설문조사 결과를 설명합니다. 설문조사 결과 재택근무를 시행하지 않은 회사의 직원 참여도가 재택근무를 시행한 회사에 비해 현저히 낮은 것으로 나타났습니다. 또한 설문조사 결과 재택근무를 선택할 수 있는 직원일수록 업무 생산성이 높고 만족도가 높다는 사실을 발견했습니다. 마지막으로 연사는 직원 참여도와 생산성을 높이기 위해 재택근무를 시행하려는 기업은 재택근무를 시행해야 한다고 결론을 내립니다.'}],
 'id': 1733}

In [166]:
qna_list = [
    "/data/llm_datasets/custom/vicuna_format/gpt_evol_1.3k-vicuna.json",
    "/data/llm_datasets/custom/vicuna_format/koalpaca_v1.1-vicuna.json",
    "/data/llm_datasets/custom/deduped/alpaca-gpt4-korean_dedup/",
    "/data/llm_datasets/custom/vicuna_format/korquad-chat-vicuna.json",
    "/data/llm_datasets/custom/vicuna_format/wizardlm_orca_vicuna.json",
    "/data/llm_datasets/sharegpt_gpt4/sharegpt_gpt4.jsonl",
    "/data/llm_datasets/custom/vicuna_format/sharegpt_V3_format_others.json",
    "/data/llm_datasets/custom/deduped/sharegpt_V3_format_ko_selected.json",
]

correction_list = [
    "/data/llm_datasets/custom/vicuna_format/KoreaSpellingCorrection/"
]

summary_list = [
    "/data/llm_datasets/custom/deduped/aihub_summary_data_tech_dedup/",
    "/data/llm_datasets/aihub_summary_data/도서/",
    "/data/llm_datasets/aihub_summary_data/법률/",
    "/data/llm_datasets/custom/deduped/naver-news-summarization-ko-vicuna_dedup/",
    
]

translation_list = [
    "/data/llm_datasets/custom/vicuna_format/sharegpt_V3_format_translation(enko).json",
    "/data/llm_datasets/custom/vicuna_format/sharegpt_V3_format_translation(koen).json",
]


dataset_list = qna_list + correction_list + summary_list + translation_list

In [564]:
dataset_path = qna_list[-1]
if dataset_path.endswith("json") or dataset_path.endswith("jsonl"):
    dataset = load_dataset("json", data_files=dataset_path)
else:
    dataset = load_dataset(dataset_path)
dataset

DatasetDict({
    train: Dataset({
        features: ['conversations', 'id'],
        num_rows: 32373
    })
})

In [571]:
train_dataset = dataset['train']

In [567]:
# alpaca-gpt4-korean_dedup
# train_dataset = dedup_by_similarity(train_dataset, prompt_template='vicuna', distance_threshold = 0.4)
# WizardLM_Orca
train_dataset, selected_ids, query_ids = dedup_by_similarity(train_dataset, prompt_template='vicuna', distance_threshold = 0.35, target_text_len=100)
# aihub_summary_data_tech_dedup
# aihub_summary_data/도서
# aihub_summary_data/법률
# naver-news-summarization-ko-vicuna_dedup
# sharegpt_V3_format_translation(enko)
# sharegpt_V3_format_translation(koen)

finished dedup data: Total:32373 Selected:277 current_dup:0 vector_store:0 remained:31465 total_dup:908


32373

In [563]:
train_dataset

Dataset({
    features: ['conversations', 'id'],
    num_rows: 3345
})

In [511]:
#train_dataset = train_dataset.select(np.random.choice(17626, 10000))
train_dataset = dedup_short(train_dataset)

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 37187/37187 [00:01<00:00, 19316.55 examples/s]

4814/37187 deduped


In [538]:
train_dataset[86]

{'conversations': [{'from': 'human',
   'value': '안녕하세요! 기술 프로그래밍 면접을 준비할 수 있도록 도와주세요. 한 번에 하나의 데이터 구조와 알고리즘 문제를 제공해 주세요. 제가 요청할 때만 힌트를 제공하세요.'},
  {'from': 'gpt',
   'value': '**문제**\n\n숫자 목록이 주어지면, 세 개의 숫자를 곱하여 만들 수 있는 가장 큰 곱셈을 반환합니다.\n\n예를 들어, 목록이 `[-10, -10, 1, 3, 2]`인 경우, `300`을 반환해야 합니다.\n\n목록에 최소 3개의 정수가 있다고 가정할 수 있습니다.\n\n**힌트**\n\n* 이 문제를 풀기 위한 한 가지 방법은 목록을 먼저 정렬한 다음 가장 큰 값과 가장 작은 값 두 가지를 고려하는 것입니다.\n* 목록에 음수가 있는 경우를 고려해야 합니다.\n\n**솔루션**\n\n이 문제를 풀기 위해 먼저 정렬된 정수 목록을 순서대로 정렬합니다. 이렇게 하면 목록에서 가장 큰 값과 가장 작은 값을 쉽게 접근할 수 있습니다.\n\n다음으로 목록의 세 번째 가장 큰 값을 고려할 수 있습니다. 정렬된 목록의 마지막 세 요소를 사용하여 이 값을 찾을 수 있습니다. 그런 다음 이 세 값의 곱을 계산하여 지금까지 최대 곱으로 저장할 수 있습니다.\n목록의 처음 두 값을 곱하여 가능한 더 큰 곱을 얻을 수 있는 경우, 목록의 가장 큰 값에 곱하여 더 큰 곱을 얻을 수 있습니다.\n마지막으로, 찾은 최대 곱을 반환하면 됩니다. 다음은 문제를 푸는 방법을 보여주는 샘플 코드입니다:```\ndef maximum_product(nums):\n    nums.sort()\n    return max(nums[-1] * nums[-2] * nums[-3], nums[0] * nums[1] * nums[-1])\n\nprint(maximum_product([-10, -10, 1, 3, 2]))  # 300\nprint(maximum_product([-5, -2, 

In [539]:
train_dataset.to_json("/workspaces/data/llm_datasets/custom/deduped2/sharegpt_V3_format_translation(koen)-00.json")

Creating json from Arrow format: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 33/33 [00:03<00:00,  9.96ba/s]


301932681

In [543]:
target_text_len = 100
def filter_question(data):
    return {
        'prompt': data['conversations'][0]['value'][:target_text_len]
    }

question_dataset = train_dataset.map(filter_question)

chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
embedder = get_embedder("ddobokki/klue-roberta-base-nli-sts-ko-en")
collection = chroma_client.create_collection(name="context", embedding_function=embedder.embed, metadata={"hnsw:space": "cosine"})
ids = []
# add
texts = question_dataset['prompt']
last_id = -1
new_ids = [f"id{i+last_id+1}" for i in range(len(texts))]
ids += new_ids
collection.add(documents=texts, ids=new_ids)

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32373/32373 [00:04<00:00, 7124.41 examples/s]


In [551]:
train_dataset[3]

{'conversations': [{'from': 'human',
   'value': '고객 세그먼트를 잘 세분화한 것인지 어떻게 알 수 있나요? 3개의 글머리 기호로 설명합니다.'},
  {'from': 'gpt',
   'value': '1. 동질성: 세그먼트는 비슷한 특성과 행동을 공유하는 고객들로 구성되어야 합니다.\n2. 고유성: 세그먼트는 고객의 특성 및 행동 측면에서 다른 세그먼트와 달라야 합니다.\n3. 안정성: 세그먼트는 시간이 지남에 따라 비교적 안정적이어야 하며 급격한 변화를 보이지 않아야 합니다. 세그먼트 내 고객의 특성과 행동은 크게 변하지 않아야 합니다.'}],
 'id': 1}

In [560]:
result = collection.query(query_texts=question_dataset[4]['prompt'], n_results=100, include=['distances', 'documents']) #

search_ids = result['ids'][0]
distances = result['distances'][0]
documents = result['documents'][0]

In [561]:
# search_ids[0]
distances, documents

([1.7881393432617188e-07,
  0.007861495018005371,
  0.05758899450302124,
  0.06374126672744751,
  0.3629910349845886,
  0.39134496450424194,
  0.40705549716949463,
  0.42446887493133545,
  0.43380725383758545,
  0.4375370740890503,
  0.4434893727302551,
  0.45005983114242554,
  0.45481860637664795,
  0.4606344699859619,
  0.46111881732940674,
  0.4629075527191162,
  0.4640594720840454,
  0.4660258889198303,
  0.467096209526062,
  0.4698142409324646,
  0.4699467420578003,
  0.4701346158981323,
  0.4732765555381775,
  0.4750330448150635,
  0.47847414016723633,
  0.48040175437927246,
  0.48438429832458496,
  0.4859827160835266,
  0.48711007833480835,
  0.4887858033180237,
  0.4891512989997864,
  0.4896743893623352,
  0.4900707006454468,
  0.490431547164917,
  0.490431547164917,
  0.4905455708503723,
  0.49115288257598877,
  0.4914326071739197,
  0.4917166829109192,
  0.49565351009368896,
  0.49570274353027344,
  0.49590861797332764,
  0.4967620372772217,
  0.4983450770378113,
  0.49922949

In [167]:
lang_distribution = {}
for dataset_path in dataset_list:
    if dataset_path.endswith("json") or dataset_path.endswith("jsonl"):
        dataset = load_dataset("json", data_files=dataset_path)
    else:
        dataset = load_dataset(dataset_path)
    
    lang_splits = {}
    for split in list(dataset.keys()):
        _dataset = dataset[split]
        
        lang_dict = {}
        for data in _dataset:
            conversations = data['conversations']
            langs = {}
            len_conv = 0
            for conv in conversations:
                _from = conv['from']
                _value = conv['value']

                len_conv += len(_value)
                lang, conf = lang_detect.predict(_value.replace('\n', ' '))
                lang = lang[0]
                if lang not in langs:
                    langs[lang] = 1
                else:
                    langs[lang] += 1

            if '__label__en' in langs:
                langs['__label__en'] -= 1
            
            if len(langs) == 0:
                dominent_lang = "empty"
            else:
                dominent_lang = max(langs)
            if dominent_lang not in lang_dict:
                lang_dict[dominent_lang] = 1
            else:
                lang_dict[dominent_lang] += 1
        lang_splits[split] = lang_dict
    lang_distribution[dataset_path] = lang_splits

In [ ]:
lang_distribution

In [168]:
with open("/workspaces/data/llm_datasets/custom/lang_distribution_SFT_v4.json", "w") as json_file:
    json.dump(lang_distribution, json_file, indent=4)

In [ ]:
with open("/workspaces/data/llm_datasets/custom/translation.json", "w") as json_file:
    json.dump(_new_dataset, json_file)

In [ ]:
# dedup_repetition
dataset_path = dataset_list[-5]
if dataset_path.endswith("json"):
    dataset = load_dataset("json", data_files=dataset_path)
else:
    dataset = load_dataset(dataset_path)
    
train_dataset = dataset['train']
data = train_dataset[0]
data

In [ ]:
_new_dataset = []
for _new_data in new_dataset:
    _new_dataset.append({
        'conversations': _new_data
    })
 

In [97]:
dataset = load_dataset("json", data_files="/workspaces/data/llm_datasets/custom/deduped/translated_sharegpt_V3_format_ko.json")
dataset

Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1064.54it/s]
Generating train split: 47187 examples [00:00, 58387.43 examples/s]


DatasetDict({
    train: Dataset({
        features: ['conversations', 'id'],
        num_rows: 47187
    })
})

In [100]:
dataset_koen = dataset['train'].select(range(10000, 15000))
dataset_enko = dataset['train'].select(range(15000, 20000))

In [112]:
dataset_koen[1]

{'conversations': [{'from': 'human',
   'value': '제품 관리자와 개발자를 포함한 사용 사례를 만들기 위한 회의 구조를 설명합니다.'},
  {'from': 'gpt',
   'value': '제품 관리자와 개발자가 포함된 사용 사례를 작성하기 위한 회의 구조는 다음과 같이 진행됩니다:1. 프로젝트의 전반적인 목표와 목적을 간략하게 검토하여 회의를 시작합니다. 이렇게 하면 모두가 개발 중인 사용 사례의 맥락과 목적을 이해하는 데 도움이 됩니다.2. 다음으로, 제품 관리자는 대상 고객과 주요 요구 사항 또는 제약 조건을 포함하여 개발 중인 제품 또는 기능에 대한 개요를 제공합니다.3. 그런 다음 개발자는 제품 또는 기능을 구현할 때 예상되는 기술적 고려 사항이나 과제, 그리고 개발 중인 사용 사례에 어떤 영향을 미칠 수 있는지에 대해 논의할 수 있습니다.4. 그런 다음 그룹은 제품 또는 기능에 대한 일반적이거나 중요한 사용 사례를 나타낼 수 있는 잠재적인 시나리오 또는 사용자 흐름을 브레인스토밍할 수 있습니다. 여기에는 사용자의 목표와 행동, 그리고 사용자가 직면할 수 있는 잠재적인 장애물이나 과제에 대한 논의가 포함될 수 있습니다.5. 그룹이 잠재적인 사용 사례를 식별하고 논의한 후에는 프로젝트 목표와 관련성이 있는 사용 사례의 우선순위를 정할 수 있습니다. 여기에는 기본 사용 사례와 보조 사용 사례를 식별하고 고려해야 할 경우의 에지 사례 또는 예외 시나리오를 식별하는 것이 포함될 수 있습니다.6. 마지막으로, 그룹은 사용 사례를 문서화하고 나머지 팀과 공유할 방법과 제품 또는 기능 개발을 진행하기 위해 필요한 다음 단계 또는 후속 조치를 논의할 수 있습니다.'},
  {'from': 'human',
   'value': '제품 관리자가 Ilford이고 개발자의 이름이 Cartoid인 예를 들어 보겠습니다.'},
  {'from': 'gpt',
   'value': '예를 들어, 제품 관리자 Ilford와 개발자 Cart

In [133]:
new_dataset = []
for data in dataset_enko:
    conversations_ko = data['conversations']
    _id = data['id']
    
    conversations_en = lang_dict['__label__en'][_id]['conversations']
    
    for _idx in range(len(conversations_ko)):
        value_ko = conversations_ko[_idx]['value']
        value_en = conversations_en[_idx]['value']
        if len(value_ko) > 150: # 너무 짧은 데이터는 품질이 좋지 않아 제거
            new_dataset.append({
                'id': f"sharegpt_V3_format_{_id}_{_idx}",
                'task': 'enkotranslation',
                'conversations': [
                                    {'from': 'human', 'value': value_en},
                                    {'from': 'gpt', 'value': value_ko},
                                 ],
            })
        
    

In [ ]:
new_dataset[0]

In [18]:
train_dataset = dataset['train']
# train_dataset = dedup_math(train_dataset)
train_dataset = dedup_repetition(train_dataset)
# train_dataset.to_json("/workspaces/data/llm_datasets/custom/deduped/aihub_summary_data_tech_dedup/train.json")

Filter: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 55102/55102 [00:54<00:00, 1003.59 examples/s]

5690/55102 deduped


In [44]:
train_dataset = dedup_too_much_token(train_dataset)

Filter: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49412/49412 [46:57<00:00, 17.54 examples/s]

2225/49412 deduped


In [ ]:
train_dataset[-25]

In [46]:
train_dataset.to_json("/workspaces/data/llm_datasets/custom/deduped/translated_sharegpt_V3_format.json")

Creating json from Arrow format: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 48/48 [00:04<00:00,  9.81ba/s]


454181195

In [ ]:
eval_dataset = dataset['test']
eval_dataset = dedup_math(eval_dataset)
eval_dataset = dedup_repetition(eval_dataset)
eval_dataset.to_json("/workspaces/data/llm_datasets/custom/deduped/aihub_summary_data_tech/test.json")

In [ ]:
len(eval_dataset)

In [17]:
def dedup_repetition(dataset):
    def validate_repetition(data):
        dedup_flag = False
        for conv in data['conversations']:
            _from = conv['from']
            _value = conv['value']

            words = _value.split(" ")
            if len(words) < 3: continue

            # unigram
            word1, word2 = words[0], words[1]
            for word3 in words[2:]:
                # print(word1, word2)
                if word1 == '' or word1 == ' ':
                    continue
                if word1 == word2 and word2 == word3:
                    dedup_flag = True
                    break
                word1, word2 = word2, word3
            if dedup_flag: 
                break
            
            continue_dup_flag = False
            if len(_value) > 50:
                for _vid in range(len(_value)-50):
                    letter = _value[_vid]
                    continue_dup_flag = True
                    for letter2 in _value[_vid:_vid+50]:
                        if letter == letter2:
                            continue
                        else:
                            continue_dup_flag = False
                            break
                    if continue_dup_flag:
                        break
                        
            if continue_dup_flag:
                dedup_flag = True
                break
            
            # bigram
            word1, word2, word3 = words[0], words[1], words[2]
            for word4 in words[3:]:
                if word1 == '' or word1 == ' ':
                    continue
                if word1 == word3 and word2 == word4:
                    dedup_flag = True
                    break
                word1, word2, word3 = word2, word3, word4
                
        # if dedup_flag:
        #     print(conv)
        
        return not dedup_flag
    
    start = len(dataset)
    dataset = dataset.filter(validate_repetition)
    print(f"{start - len(dataset)}/{start} deduped")
    return dataset

In [ ]:
def dedup_math(dataset):
    def validate_math(data):
        dedup_flag = False
        for conv in data['conversations']:
            _from = conv['from']
            _value = conv['value']
            words = _value.split("\\")
            if len(words) > 3:
                dedup_flag = True
                break

        # if dedup_flag:
        #     print(conv)
        
        return not dedup_flag
    
    start = len(dataset)
    dataset = dataset.filter(validate_math)
    print(f"{start - len(dataset)}/{start} deduped")
    return dataset

In [43]:
def dedup_too_much_token(dataset):
    def validate_too_much_token(data):
        dedup_flag = False
        for conv in data['conversations']:
            _from = conv['from']
            _value = conv['value']
            
            input_json = {
                "model_name": "MingAI-70B-chat-orca_v0.42_2_dpo-GPTQ",
                "prompt": _value,
            }

            ret = requests.post(api_server_url + "/count_token", json=input_json)

            output_json = ret.json()
            num_token += output_json['count']
            
        if num_token > 3500:
            dedup_flag = True
            break

        # if dedup_flag:
        #     print(conv)
        
        return not dedup_flag
    
    start = len(dataset)
    dataset = dataset.filter(validate_too_much_token)
    print(f"{start - len(dataset)}/{start} deduped")
    return dataset

In [ ]:
# for KOR_platypus
def dedup_python(dataset):
    def validate_python(data):
        dedup_flag = False
        value = data['conversations'][1]['value']
        if 'python' in value or 'def ' in value:
            dedup_flag = True
        return not dedup_flag
    
    start = len(dataset)
    dataset = dataset.filter(validate_python)
    print(f"{start - len(dataset)}/{start} deduped")
    return dataset

In [ ]:
# for KOR_platypus
def add_python_instruct(data):
    
    return {
        'id': data['id'],
        'instruction': "",
        'conversations',
        'task_name': 'instruct',
    }
    
    
    ['instruction', 'id', 'conversations', 'task_name']

In [507]:
# for sharegpt_ko
def dedup_short(dataset):
    def validate_short(data):
        dedup_flag = False
        for conv in data['conversations']:
            _from = conv['from']
            _value = conv['value']
            
            if len(_value) < 10:
                dedup_flag = True
                break

        # if dedup_flag:
        #     print(conv)
        
        return not dedup_flag
    
    start = len(dataset)
    dataset = dataset.filter(validate_short)
    print(f"{start - len(dataset)}/{start} deduped")
    return dataset

In [ ]:
train_dataset.to_json("/workspaces/data/llm_datasets/custom/deduped/aihub_summary_data_tech/train.json")

In [ ]:
with open("/workspaces/data/llm_datasets/custom/deduped/aihub_summary_data_tech", "w") as json_file:
    json.dump(train_dataset, json_file)

In [ ]:
# dedup_wrong_code

In [ ]:
# refine_code_block ```pythondef -> ```python\ndef

In [2]:
import chromadb
from chromadb.config import Settings
import random
from fastchat.modules.embedder_adapter import Embedder, get_embedder

In [ ]:
chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
embedder = get_embedder("ddobokki/klue-roberta-base-nli-sts-ko-en")
collection = chroma_client.create_collection(name="context", embedding_function=embedder.embed, metadata={"hnsw:space": "cosine"})
ids = []

In [ ]:
len_system = len('### System:\nThis is a system prompt, please behave and help the user.')
len_user = len('\n\n### User: ')
len_bot = len('\n\n\n### Assistant:')
text_len = 100
def remove_system(data):
    return { 
        **data,
        'prompt': data['prompt'][len_system + len_user:-len_bot][:text_len]
    }

eval_dataset = eval_dataset.map(remove_system)

In [ ]:
eval_dataset[30]

In [ ]:
# add
texts = eval_dataset['prompt']

last_id = -1
new_ids = [f"id{i+last_id+1}" for i in range(len(texts))]
ids += new_ids
collection.add(documents=texts, ids=new_ids)

In [ ]:
eval_dataset['prompt'][0][:]

In [ ]:
len(new_ids)

In [ ]:
import copy
query_ids = copy.deepcopy(new_ids)

n_results = 100
threshold = 0.6

selected_ids = []
duplicated_ids = []
# while query_ids:

In [ ]:
while query_ids:
    current_id = random.choice(query_ids)
    selected_ids.append(current_id)
    search_strings = [texts[int(current_id[2:])]]
    if collection.count() == 0:
        print("Warning: collection is empty. Forced break")
        break
    result = collection.query(query_texts=search_strings, n_results=min(n_results, len(query_ids)), include=['distances']) #'documents'

    search_ids = result['ids'][0]
    distances = result['distances'][0]
    remove_ids = []
    for idx in range(len(search_ids)):
        sid = search_ids[idx]
        dist = distances[idx]
        if dist < threshold:
            remove_ids.append(sid)
    
    for rid in remove_ids:
        if rid in query_ids:
            query_ids.remove(rid)
    duplicated_ids += remove_ids
    collection.delete(ids=remove_ids)
    
    print(len(new_ids), len(selected_ids), len(remove_ids), collection.count(), len(query_ids), len(duplicated_ids), '\t\t\t\t\t', end='\r')
    

In [ ]:

file_paths = glob.glob("/workspaces/data/llm_datasets/aihub/*[!tar|!sh]")

dataset_dict = {}
for file_path in file_paths:
    file_name = os.path.basename(file_path)
    
    new_dataset = []
    idx = 0

    paths = glob.glob(os.path.join(file_path, '*.json'))
    
    for path in paths:
        with open(path, "r") as f:
            json_data = json.load(f)

        context_info = json_data['dataset']['context_info']
        for context_data in context_info:
            context = context_data['context']
            summary = context_data['summary']

            data_row = {
                'id': f"{file_name}_{idx}",
                'task': 'summarization',
                'conversations': [
                                    {'from': 'human', 'value': context},
                                    {'from': 'gpt', 'value': summary},
                                 ],
            }
            new_dataset.append(data_row)
            idx += 1
        
    print(f"file_name:{file_name} idx:{idx}", '\t\t\t\t\t\t', end='\r')
    dataset_dict[file_name] = new_dataset

In [ ]:
train_dataset_list = dataset_dict['TL_EE_train'] + dataset_dict['TL_LA_train'] + dataset_dict['TL_ED_train'] + dataset_dict['TL_NA_train']
eval_dataset_list = dataset_dict['TL_EE_val'] + dataset_dict['TL_LA_val'] + dataset_dict['TL_ED_val'] + dataset_dict['TL_NA_val']

In [ ]:
with open("/workspaces/data/llm_datasets/aihub_summary_data/train.json", "w") as json_file:
    json.dump(train_dataset_list, json_file)
with open("/workspaces/data/llm_datasets/aihub_summary_data/test.json", "w") as json_file:
    json.dump(eval_dataset_list, json_file)

In [ ]:
dataset = load_dataset("/workspaces/data/llm_datasets/aihub_summary_data")

In [ ]:

file_paths = glob.glob("/workspaces/data/llm_datasets/aihub/*summary*[!tar|!sh]")

dataset_dict = {}
for file_path in file_paths:
    file_name = os.path.basename(file_path)
    
    new_dataset = []
    idx = 0

    paths = glob.glob(os.path.join(file_path, '*.json'))
    
    for path in paths:
        with open(path, "r") as f:
            json_data = json.load(f)

        documents = json_data['documents']
        for document in documents:
            text = document['text']
            abstractive = document['abstractive']

            summary = abstractive[0]
            context = []
            for _text in text:
                _context = ' '.join([_index_text['sentence'] for _index_text in _text])
                context.append(_context)
            context = '\n'.join(context)
            
            data_row = {
                'id': f"{file_name}_{idx}",
                'task': 'summarization',
                'conversations': [
                                    {'from': 'human', 'value': context},
                                    {'from': 'gpt', 'value': summary},
                                 ],
            }
            new_dataset.append(data_row)
            idx += 1
        
        print(f"file_name:{file_name} idx:{idx}", '\t\t\t\t\t\t', end='\r')
    dataset_dict[file_name] = new_dataset

In [ ]:
for key in dataset_dict.keys():
    print(key, len(dataset_dict[key]))

In [ ]:
train_dataset_list = dataset_dict['summary_law_train']
eval_dataset_list = dataset_dict['summary_law_val']

In [ ]:
with open("/workspaces/data/llm_datasets/aihub_summary_data/법률/train.json", "w") as json_file:
    json.dump(train_dataset_list, json_file)
with open("/workspaces/data/llm_datasets/aihub_summary_data/법률/test.json", "w") as json_file:
    json.dump(eval_dataset_list, json_file)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("/workspaces/data/llm_datasets/aihub_summary_data/법률")
dataset

In [ ]:
dataset['test'][0]

In [ ]:
file_paths = glob.glob("/workspaces/data/llm_datasets/aihub/*summary_book*[!tar|!sh]")
file_paths

In [ ]:
file_path = file_paths[0]
paths = glob.glob(os.path.join(file_path, '**/*.json'))
paths[0], len(paths)

In [ ]:
path = paths[0]
with open(path, "r") as f:
    json_data = json.load(f)

In [ ]:
json_data.keys()

In [ ]:
context = json_data['passage']
summary = json_data['summary']

In [ ]:

summary

In [ ]:

file_paths = glob.glob("/workspaces/data/llm_datasets/aihub/*summary_book*[!tar|!sh]")

dataset_dict = {}
for file_path in file_paths:
    file_name = os.path.basename(file_path)
    
    new_dataset = []
    idx = 0

    paths = glob.glob(os.path.join(file_path, '**/*.json'))
    
    for path in paths:
        with open(path, "r") as f:
            json_data = json.load(f)

        context = json_data['passage']
        summary = json_data['summary']
        
        data_row = {
            'id': f"{file_name}_{idx}",
            'task': 'summarization',
            'conversations': [
                                {'from': 'human', 'value': context},
                                {'from': 'gpt', 'value': summary},
                             ],
        }
        new_dataset.append(data_row)
        idx += 1
        
        print(f"file_name:{file_name} idx:{idx}", '\t\t\t\t\t\t', end='\r')
    dataset_dict[file_name] = new_dataset

In [ ]:
for key in dataset_dict.keys():
    print(key, len(dataset_dict[key]))

In [ ]:
train_dataset_list = dataset_dict['summary_book_train']
eval_dataset_list = dataset_dict['summary_book_val']

In [ ]:
with open("/workspaces/data/llm_datasets/aihub_summary_data/도서/train.json", "w") as json_file:
    json.dump(train_dataset_list, json_file)
with open("/workspaces/data/llm_datasets/aihub_summary_data/도서/test.json", "w") as json_file:
    json.dump(eval_dataset_list, json_file)

In [ ]:
dataset = load_dataset("/workspaces/data/llm_datasets/aihub_summary_data/도서")
dataset

In [ ]:
dataset['train'][1]

In [ ]:
qas

In [ ]:

file_paths = glob.glob("/workspaces/data/llm_datasets/aihub/*VL*[!tar|!sh]")

dataset_dict = {}
for file_path in file_paths:
    file_name = os.path.basename(file_path)
    
    new_dataset = []
    idx = 0

    paths = glob.glob(os.path.join(file_path, '*.json'))
    
    for path in paths:
        with open(path, "r") as f:
            json_data = json.load(f)

        context_info = json_data['dataset']['context_info']
        for context_data in context_info:
            context = context_data['context']
            qas = context_data['qas']

            for _qas in qas:
                question = _qas['question-1']
                answer = _qas['answer']
                question_level = _qas['question_level']
                if question_level != '상': continue
                data_row = {
                    'id': f"{file_name}_{idx}",
                    'task': 'contextqa',
                    'context': context,
                    'question': question,
                    'answer': answer,
                }
                new_dataset.append(data_row)
                idx += 1
        
        print(f"file_name:{file_name} idx:{idx}", '\t\t\t\t\t\t', end='\r')
    dataset_dict[file_name] = new_dataset

In [ ]:
for key in dataset_dict.keys():
    print(key, len(dataset_dict[key]))

In [ ]:
train_dataset_list = dataset_dict['VL_EE_train'] + dataset_dict['VL_NA_train'] + dataset_dict['VL_LA_train']+ dataset_dict['VL_ED_train']
eval_dataset_list = dataset_dict['VL_EE_val'] + dataset_dict['VL_NA_val'] + dataset_dict['VL_LA_val']+ dataset_dict['VL_ED_val']

In [ ]:
with open("/workspaces/data/llm_datasets/aihub_contextqa_data_hard/기술과학/train.json", "w") as json_file:
    json.dump(train_dataset_list, json_file)
with open("/workspaces/data/llm_datasets/aihub_contextqa_data_hard/기술과학/test.json", "w") as json_file:
    json.dump(eval_dataset_list, json_file)

In [ ]:
train_dataset_list_0 = train_dataset_list[:120000]
train_dataset_list_1 = train_dataset_list[120000:240000]
train_dataset_list_2 = train_dataset_list[240000:]

with open("/workspaces/data/llm_datasets/aihub_contextqa_data/기술과학/train_split0.json", "w") as json_file:
    json.dump(train_dataset_list_0, json_file)
    
with open("/workspaces/data/llm_datasets/aihub_contextqa_data/기술과학/train_split1.json", "w") as json_file:
    json.dump(train_dataset_list_1, json_file)
    
with open("/workspaces/data/llm_datasets/aihub_contextqa_data/기술과학/train_split2.json", "w") as json_file:
    json.dump(train_dataset_list_2, json_file)

In [ ]:
dataset = load_dataset("/workspaces/data/llm_datasets/aihub_contextqa_data_hard/기술과학")
dataset

In [ ]:
dataset['train'][0]

In [ ]:
dataset = load_dataset("/workspaces/data/llm_datasets/gpt4_evol_1.3k/data/")

In [ ]:
data = dataset['train'][0]
# answer = data['answer']
# question = data['question']
data

In [ ]:
new_dataset = []
idx = 0
for data in dataset['train']:
    answer = data['answer']
    question = data['question']

    data_row = {
        'id': f"gpt_evol_1.3k_{idx}",
        'conversations': [
                            {'from': 'human', 'value': question},
                            {'from': 'gpt', 'value': answer},
                         ],
    }
    new_dataset.append(data_row)
    idx += 1

In [ ]:
new_dataset[2]

In [ ]:
with open("/data/llm_datasets/custom/vicuna_format/gpt_evol_1.3k-vicuna.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [ ]:
dataset = load_dataset("json", data_files="/data/llm_datasets/WizardLM_Orca/wizardlm_orca.json")

In [ ]:
new_dataset = []
idx = 0
for data in dataset['train']:
    output = data['output']
    system = data['system']
    instruction = data['instruction']
    data_row = {
        'id': f"WizardLM_Orca_{idx}",
        'conversations': [
                            {'from': 'human', 'value': instruction},
                            {'from': 'gpt', 'value': output},
                         ],
        'task': 'system_instruct',
        'system': system,
    }
    new_dataset.append(data_row)
    idx += 1
    

In [ ]:
with open("/data/llm_datasets/custom/vicuna_format/wizardlm_orca_vicuna.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [ ]:
dataset = load_dataset("/data/llm_datasets/KoreaSpellingCorrection/")

In [ ]:
new_dataset = []
idx = 0
for data in dataset['test']:
    wrong = data['wrong']
    correct = data['correct']
    data_row = {
        'id': f"KoreaSpelling_Correction_{idx}",
        'conversations': [
                            {'from': 'human', 'value': wrong},
                            {'from': 'gpt', 'value': correct},
                         ],
        'task': 'correction',
    }
    new_dataset.append(data_row)
    idx += 1

In [ ]:
new_dataset[2]

In [ ]:
with open("/data/llm_datasets/custom/vicuna_format/KoreaSpellingCorrection/test.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [ ]:
dataset = load_dataset("json", data_files="/data/llm_datasets/alpaca-gpt4-korean/alpaca-gpt4-korean.json")

In [ ]:
dataset['train'][10]

In [ ]:
for data in dataset['train']:
    conversations = data['conversations']
    if len(conversations) != 2:
        print("!")

In [ ]:
dataset = load_dataset("json", data_files="/data/llm_datasets/custom/deduped/alpaca-gpt4-korean_dedup/train.json")

In [ ]:
dataset['train'][5]

In [ ]:
dataset = load_dataset("/workspaces/data/llm_datasets/aihub_contextqa_data_hard/기술과학")

In [ ]:
%%time
def send_request(new_dataset):
    global idx
    for _ in range(2):
        if idx > len_dataset:
            break
        lock.acquire()
        data = dataset[subset][idx]
        idx += 1
        lock.release()
        
        print(f"{idx}/{len_dataset}", '\t\t\t\t\t\t', end='\r')
        
        _id = data['id']
        context = data['context']
        question = data['question']
        answer = data['answer']
        if answer.lower() == 'yes':
            answer = '네'
        elif answer.lower() == 'no':
            answer = '아니오'
        
        # response
        result = generate_refiner(
            model_name,
            context,
            question,
            answer
        )
        
        new_dataset.append({
            'id': _id,
            'conversations': [
                                {'from': 'human', 'value': question},
                                {'from': 'gpt', 'value': result},
                             ],
            'task_name': "instruct",
            'instruction': context,
        })


model_name = "MingAI-70B-chat-orca_v0.42_2_dpo-GPTQ"
subset = 'train'
new_dataset = []
threads = []
idx = 0
lock = threading.Lock()
len_dataset = len(dataset[subset])
n_thread = 64


for i in range(n_thread):
    t = threading.Thread(target=send_request, args=(new_dataset,)) # 
    t.start()
    # time.sleep(0.5)
    threads.append(t)
    
for t in threads:
    t.join()

In [ ]:
file_paths = glob.glob("/workspaces/data/llm_datasets/aihub/*translation_tech*[!tar|!sh]")
file_path = file_paths[0]

In [ ]:
paths = glob.glob(os.path.join(file_path, '*.json'))
paths[0]

In [ ]:
path = paths[0]
with open(path, "r") as f:
    json_data = json.load(f)

In [ ]:
data = json_data['data']

In [ ]:
_data = data[1010]

In [ ]:
_data

In [74]:
dataset = load_dataset("json", data_files="/data/llm_datasets/sharegpt_gpt4/sharegpt_gpt4.jsonl")
dataset

DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 6206
    })
})

In [87]:
dataset['train'][15]

{'conversations': [{'from': 'human',
   'value': '【발명의 설명】\n【발명의 명칭】\n활동정보 기반 회원 등급결정 플랫폼 운용 서버 및 그 동작 방법{Activity information-based member rating determination platform operation server and operation method thereof}\n【기술분야】\n본 발명은 활동정보 기반 회원 등급결정 플랫폼 운용 서버 및 그 동작 방법에 관한 것으로, 더욱 상세하게는 회원의 다양한 활동 정보에 따라 회원 등급을 결정할 수 있는 서버 및 그 동작 방법에 관한 것이다.\n\n【발명의 배경이 되는 기술】\n최근 비대면 플랫폼 기술의 발달로 많은 사람들은 대면하지 않고 온라인을 통한 서비스를 제공받을 수 있게 되었다. 제공되는 서비스는 점차 다양화되고 확대되면서 온라인 쇼핑몰, 동영상 스트리밍 또는 다양한 커뮤니티 등에서 비대면 서비스를 제공받을 수 있게 되었다.\n이때, 서비스를 제공받고 자신의 온라인 활동에 대한 신용도를 축적하기 위해 회원으로 가입해야 하는 경우가 다수이며, 이를 통해 사용자들은 회원으로서의 혜택을 향유할 수 있다.\n회원은 단순히 온라인 상의 특정 사용자를 지칭하는 것을 넘어 특정 등급에 따라 여러가지 혜택을 제공받을 수 있는데, 기존의 회원 등급은 온라인 서비스 운영자의 미리 설정된 기준에 따라 정하여 지거나, 다수의 이익보다는 특정 집단(예: 상위 등급의 회원 또는 서비스 운영자)의 이익에 중점을 두고 있는 실정이다.\n서비스를 제공하고 그 대가로부터 이익을 발생시키는 상품의 경우, 그 상품의 성질을 떠나 해당 상품을 제공하기 위한 상품의 원가와 소비자가 해당 상품을 획득하기 위해 지불해야 하는 대가 사이에 차이가 발생하며, 서비스의 운영자는 그 차이를 조정함으로써 자신 또는 다른 사용자의 이익을 발생시킬 수 있다.\n그러나, 이러한 혜택을 각 회원의 활동 또는 각 플랫폼의 운영체계에 의해 공평한 대가

In [95]:
dataset = load_dataset("json", data_files="/data/llm_datasets/lima_vicuna_format/lima_vicuna_format.json")
enko_dataset = []
remained_dataset = []

lang_dict = {}
for data in dataset['train']:
    conversations = data['conversations']
    langs = {}
    len_conv = 0
    for conv in conversations:
        _from = conv['from']
        _value = conv['value']

        len_conv += len(_value)
        lang, conf = lang_detect.predict(_value.replace('\n', ' '))
        lang = lang[0]
        if lang not in langs:
            langs[lang] = 1
        else:
            langs[lang] += 1

    if '__label__en' in langs:
        langs['__label__en'] -= 1

    dominent_lang = max(langs)
    if dominent_lang not in lang_dict:
        lang_dict[dominent_lang] = [data]
    else:
        lang_dict[dominent_lang].append(data)
    # break

In [96]:
for key, value in lang_dict.items():
    print(key, len(value))

__label__en 55102
__label__fr 384
__label__es 386
__label__km 508
__label__id 51
__label__ur 6
__label__ko 470
__label__zh 289
__label__it 224
__label__pt 208
__label__vi 52
__label__ro 24
__label__nl 70
__label__hu 99
__label__sv 84
__label__fi 23
__label__nds 5
__label__pa 9
__label__ja 89
__label__ru 107
__label__pl 55
__label__war 5
__label__tl 11
__label__he 23
__label__fa 26
__label__kn 11
__label__sh 18
__label__tr 56
__label__no 44
__label__sl 5
__label__tg 1
__label__ne 4
__label__et 9
__label__la 17
__label__uz 5
__label__oc 3
__label__gl 5
__label__mr 4
__label__sw 2
__label__ms 17
__label__sr 17
__label__fy 3
__label__te 6
__label__uk 33
__label__hr 8
__label__th 11
__label__new 1
__label__hi 11
__label__eo 15
__label__my 1
__label__sk 4
__label__ga 2
__label__tt 5
__label__gu 1
__label__nn 1
__label__de 5
__label__lmo 3
__label__ta 4
__label__is 1
__label__hy 1
__label__ml 1
__label__eu 6
__label__lt 2
__label__jbo 1
__label__io 2
__label__wuu 1
__label__mk 2
__label__ia 2

In [94]:
lang_dict['__label__ko'][201]

{'conversations': [{'from': 'human',
   'value': '당신은 EMDR이라는 정신과에서 PTSD환자들을 위해서 고안한 치료법을 앱으로 구현하려고 기획중입니다. 눈을 좌우로 움직이면서 지워지지않는 생각을 상쇄시키는 효과를 볼 수 있습니다. 이러한 눈움직이기 활동을 모바일 화면을 보면서 수행할 수 있도록 활동을 기획할때 좌우로 움직이는 것뿐 아니라, 좌우, 위아래, 왼쪽 회전 오른쪽 회전 등의 활동을 30초씩 진행하고자할때 어떤 식으로 기획서를 작성할 수 있을지 아이디어를 제시하십시오'},
  {'from': 'gpt',
   'value': '기획서 작성에 필요한 주요 요소들을 몇 가지 제시해 드리겠습니다. 이러한 요소들을 기반으로 기획서를 작성하시면, EMDR 앱을 효과적으로 구현할 수 있을 것입니다.\n\n1. 프로젝트 개요: EMDR 앱을 이용하여 PTSD 환자들에게 치료 효과를 제공할 수 있는 모바일 환경의 EMDR 치료법을 구현하고자 함을 명시합니다.\n2. 목표 및 목적: 앱의 주요 목표는 PTSD 환자들이 스스로 치료를 진행할 수 있는 모바일 치료 도구를 제공하는 것입니다. 또한, 환자들이 눈 움직임을 좌우, 상하, 회전 등 다양한 방식으로 진행할 수 있도록 기능을 구현합니다.\n3. 기능 및 요구 사항:\n* 눈 움직임 유도 화면: 화면 상에 눈을 따라 움직이는 물체를 표시하여 사용자가 눈 움직임을 쉽게 따라할 수 있도록 합니다. 좌우, 상하, 왼쪽 회전, 오른쪽 회전 등의 다양한 움직임을 구현해야 합니다.\n* 타이머 기능: 각 움직임에 대해 30초씩 진행할 수 있는 타이머를 제공합니다.\n* 사용자 설정: 사용자가 원하는 눈 움직임 조합과 순서, 시간을 설정할 수 있는 기능을 구현합니다.\n* 진행 기록 및 피드백: 사용자가 진행한 치료 세션의 기록을 저장하고, 그에 따른 피드백을 제공합니다.\n1. 기술 및 도구: 앱 개발에 필요한 기술 스택과 도구를 명시합니다. 예를 들어, Android와 iOS 플랫폼을 

In [ ]:
import requests
api_server_url = "http://localhost:41002"
def count_total_token(conversations):
    num_token = 0
    for conv in conversations:
        input_json = {
            "model_name": "MingAI-70B-chat-orca_v0.42_2_dpo-GPTQ",
            "prompt": conv['value'],
        }

        ret = requests.post(api_server_url + "/count_token", json=input_json)

        output_json = ret.json()
        num_token += output_json['count']
    return num_token


for _idx in range(10):
    num_total_token = count_total_token(lang_dict['__label__en'][_idx])
    print(num_total_token)

In [141]:
len(new_dataset)

3572

In [140]:
new_dataset = []
for key, item in lang_dict.items():
    if key != '__label__en':
        new_dataset += item
        

In [ ]:
%%time
api_server_url = "http://localhost:41002"
def send_translate_request(new_dataset):
    global idx
    # for _ in range(2):
    while(1):
        if idx > len_dataset:
            break
        lock.acquire()
        pidx = idx
        data = lang_dict['__label__en'][pidx]
        idx += 1
        lock.release()
        
        print(f"{idx}/{len_dataset}", '\t\t\t\t\t\t', end='\r')#
        
        conv = data['conversations']
        new_conv = []
        for _data in conv:
            value = _data['value']
            # response
            results = ""
            text_blocks = []
            code_blocks = []
            for bidx, block in enumerate(value.split("```")):
                if bidx % 2 == 0:
                    text_blocks.append(block)
                else:
                    code_blocks.append(block)

            for tidx, text_block in enumerate(text_blocks):
                prompt = f"### 영어:\n{text_block}\n### 한국어:\n"
                input_json = {
                    "model_name": "Gugugo-koen-7B-V1.1",
                    "prompt": prompt,
                    "temperature": 0.7,
                    "top_p": 0.8,
                    "max_new_tokens": 4096,
                    "stop": ["</끝>", "###"],
                }

                ret = requests.post(
                    api_server_url + "/worker_generate_stream",
                    json=input_json,
                    stream=True,
                )

                for chunk in ret.iter_lines(decode_unicode=False, delimiter=b"\0"):
                    if chunk:
                        result_data = json.loads(chunk.decode())

                result = result_data['text'][len(prompt):].rstrip('\n')
                results += result
                if len(code_blocks) > tidx:
                    results += "```" + code_blocks[tidx] + "```"
            
            new_conv.append({
                'from': _data['from'],
                'value': results,
            })
        new_dataset.append({
            'conversations': new_conv,
            'id': pidx,
        })


# code_prefixes = ["python", "c++", "minikube", "docker", "json", "java", 
#                  "php", "bash", "c#", "cpp", "css", "perl", "html", "xml", 
#                  "ruby", "sql", "ini", "apt", "socat", "tcp", "localhost",
#                  "git"]
new_dataset = []
threads = []
idx = 0
lock = threading.Lock()
len_dataset = len(lang_dict['__label__en'])
n_thread = 64 * 7


for i in range(n_thread):
    t = threading.Thread(target=send_translate_request, args=(new_dataset,)) # 
    t.start()
    # time.sleep(0.5)
    threads.append(t)
    
for t in threads:
    t.join()

In [ ]:
new_dataset[-3]

In [11]:
with open("/data/llm_datasets/custom/vicuna_format/translated_sharegpt_V3_format.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [ ]:
nidx=10

In [ ]:
lang_dict['__label__en'][40] #코드처리 ```, "", (""), {}, 반복처리, 빈 데이터(길이 이슈) 제거

In [ ]:
results = ""
text_blocks = []
code_blocks = []
for bidx, block in enumerate(value.split("```")):
    if bidx % 2 == 0:
        text_blocks.append(block)
    else:
        code_blocks.append(block)

for tidx, text_block in enumerate(text_blocks):
    prompt = f"### 영어:\n{text_block}\n### 한국어:\n"
    input_json = {
        "model_name": "Gugugo-koen-7B-V1.1",
        "prompt": prompt,
        "temperature": 0.7,
        "top_p": 0.8,
        "max_new_tokens": 4096,
        "stop": ["</끝>", "###"],
    }

    ret = requests.post(
        api_server_url + "/worker_generate_stream",
        json=input_json,
        stream=True,
    )

    for chunk in ret.iter_lines(decode_unicode=False, delimiter=b"\0"):
        if chunk:
            result_data = json.loads(chunk.decode())
    result = result_data['text'][len(prompt):].rstrip('\n')
    results += result

    if len(code_blocks) > tidx:
        results += "```" + code_blocks[tidx] + "```"
results

In [ ]:
dataset2 = load_dataset("json", data_files="/data/llm_datasets/sharegpt_gpt4/sharegpt_V3_format.jsonl")
dataset2

In [ ]:
dataset2['train'][1]

In [ ]:
dataset = load_dataset("json", data_files="/data/llm_datasets/custom/vicuna_format/wizardlm_orca_vicuna.json")

In [ ]:
dataset['train'][31]

In [ ]:
train_dataset[-9]